In [1]:
import requests as rq
import ujson as js
import pandas as pd
from datetime import datetime as dt

from ApiHandler import ApiHandler

In [2]:
class ApiSystemStats(ApiHandler):
    settings = {
        **ApiHandler.settings,
        'data_timestamp': 'jumps_data'
    }
    url = {
        **ApiHandler.url,
        'jumps_data': 'https://esi.evetech.net/latest/universe/system_jumps/',
        'kills_data': 'https://esi.evetech.net/latest/universe/system_kills/'
    }
    script_vals = {
        **ApiHandler.script_vals,
        'table': 'SystemStats'
    }
    name = 'ESI System Stats API'
    timestamp = {}
    
    def get_raw_data(self):
        if self.verbose: self._verbose('get_raw_data', 'Getting raw data for date {date}...'.format(**{'date':data_date}))
        
        raw_data = {
            'jumps_data': self.get_data_item('jumps_data'),
            'kills_data': self.get_data_item('kills_data')
        }
        
        return raw_data
    
    def get_data_item(self, conn_type):
        data_conn = rq.get(self.url[conn_type])
        
        if data_conn.status_code != 200:
            raise Exception("""\
                Connection {conn_type} returned a {status} code on pull for {date}.
                Message body:
                {body}\
            """.format(**{
                'conn_type':conn_type,
                'status':data_conn.status_code,
                'date': date,
                'body':data_conn.content.decode('utf-8')
            }))
        
        self.timestamp[conn_type] = dt.strptime(
            data_conn.headers['Last-Modified'],
            '%a, %d %b %Y %H:%M:%S %Z'
        ).strftime('%Y-%m-%d %H:%M:%S')
        
        return data_conn.content
    
    def build_data(self, raw_data):
        if self.verbose: self._verbose('build_data', 'Building data frame...')
        
        data_frame = pd.DataFrame(js.loads(raw_data['jumps_data'])).set_index('system_id').join(
            pd.DataFrame(js.loads(raw_data['kills_data'])).set_index('system_id'),
            how = 'outer'
        ).fillna(0).reset_index()
        data_frame.insert(0, 'record_time', self.timestamp[self.settings['data_timestamp']])
        
        if self.verbose: self._verbose('build_data', 'Data frame built. %s records.' % len(data_frame))
            
        return data_frame

In [4]:
api = ApiSystemStats()
api.run_process()